# max_div_seq Runner

In [7]:
TIME_LIMIT = 10
SEMESTER = '2017B'

In [8]:
import pybryt
import os
import signal
from contextlib import contextmanager
from shutil import copy, rmtree
from tqdm import tqdm
import IPython.nbformat as nbf
from pathlib import Path

In [9]:
ref = pybryt.ReferenceImplementation.compile("max_even_seq.ipynb")
ref.dump("max_even_seq-ref.pkl")
ref = pybryt.ReferenceImplementation.load("max_even_seq-ref.pkl")

In [10]:
from glob import glob
subs = glob(f'subs/{SEMESTER}/*.py')

In [11]:
notebooks = []

for sub in subs:

    code = Path(sub).read_text()

    cells = []

    cell = nbf.v4.new_code_cell(source=code)
    cells.append(cell)
    cell = nbf.v4.new_code_cell(source="max_even_seq(222)")
    cells.append(cell)

    nb = nbf.v4.new_notebook(cells = cells)

    pre, ext = os.path.splitext(sub)
    notebook = pre + '.ipynb'
    with open(notebook, 'w') as f:

            nbf.write(nb, f, 4)
    notebooks.append(notebook)

In [12]:
class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(_, __):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)

student_impls = []
for sub in tqdm(notebooks):
    try:
        with time_limit(TIME_LIMIT):
            student_impls.append(pybryt.StudentImplementation(sub))
    except Exception as e:
        print(f'{type(e).__name__} in {sub}')
        student_impls.append(None)

 10%|█         | 20/198 [00:40<12:32,  4.23s/it]

TimeoutException in subs/2017B/11.ipynb


 16%|█▌        | 31/198 [01:00<05:19,  1.92s/it]

SyntaxError in subs/2017B/65.ipynb
ValueError in subs/2017B/51.ipynb


 30%|███       | 60/198 [01:56<11:20,  4.93s/it]

...

 96%|█████████▌| 190/198 [05:27<00:13,  1.64s/it]

SyntaxError in subs/2017B/94.ipynb


 97%|█████████▋| 193/198 [05:30<00:06,  1.39s/it]

SyntaxError in subs/2017B/12.ipynb


100%|██████████| 198/198 [05:37<00:00,  1.70s/it]


In [13]:
results = []
for si in student_impls:
    if si is None:
        results.append(None)
    else:
        results.append(si.check(ref))

In [14]:
path = f'./advices/{SEMESTER}'
if os.path.exists(path):
    rmtree(path)
os.makedirs(path)
for sp, res in zip(subs, results):
    if res is None:
        os.makedirs(f'{path}/Error', exist_ok=True)
        copy(sp, f'{path}/Error/')
        continue
    if res.correct:
        os.makedirs(f'{path}/None', exist_ok=True)
        copy(sp, f'{path}/None/')
        continue
    for message in res.messages:
        os.makedirs(f'{path}/{message}', exist_ok=True)
        copy(sp, f'{path}/{message}/')

in
